In [ ]:
!pip install pytesseract easyocr
!apt-get install tesseract-ocr
!pip install --upgrade scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 13.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2

In [ ]:
import matplotlib.pyplot as plt
import os
import zipfile
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# Fonction pour supprimer un fichier/dossier s'il existe
def remove_if_exists(path):
    if os.path.exists(path):
        if os.path.isdir(path):
            os.rmdir(path)  # Supprimer un dossier
        else:
            os.remove(path)  # Supprimer un fichier
        print(f"{path} a été supprimé.")
    else:
        print(f"{path} n'existe pas.")

extracted_etiquette_folder = '/content/drive/MyDrive/Project/Deep Learning/cropped_labels'


Mounted at /content/drive


#Fonctions

In [ ]:
# Fonction pour afficher avant et après
def display_before_after(before, after, title_before, title_after):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(before, cmap='gray')
    plt.title(title_before)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(after, cmap='gray')
    plt.title(title_after)
    plt.axis('off')

    plt.show()


##Deskew

In [ ]:
import cv2
import numpy as np

def deskew_image(image):
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un flou pour lisser l'image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Détecter les bords avec Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Détection des lignes avec Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    if lines is not None:
        # Calculer l'angle moyen des lignes détectées
        angles = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Calculer l'angle de chaque ligne
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            angles.append(angle)

        # Calculer la médiane des angles
        angle = np.median(angles)

    else:
        angle = 0  # Si aucune ligne n'est détectée, pas de rotation nécessaire

    # Appliquer la rotation pour corriger l'inclinaison
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated_image


##Binarisation

In [ ]:
import cv2
import numpy as np
from skimage.filters import threshold_sauvola

def sauvola_binarization(image, window_size=25, k=0.2, R=128):
    """
    Applique la binarisation de Sauvola à une image en utilisant scikit-image.

    Parameters:
        image (numpy array): L'image à traiter (en niveaux de gris ou couleur).
        window_size (int): Taille de la fenêtre pour calculer le seuil local.
        k (float): Facteur de pondération pour la variance locale.
        R (float): Paramètre de normalisation (par défaut 128, recommandé par Sauvola).

    Returns:
        numpy array: Image binarisée.
    """
    # Convertir l'image en niveaux de gris si nécessaire
    if len(image.shape) == 3:  # Si l'image est en couleur
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Calculer le seuil local en utilisant Sauvola
    sauvola_thresh = threshold_sauvola(gray_image, window_size=window_size, k=k, r=R)

    # Appliquer la binarisation
    binarized_image = (gray_image > sauvola_thresh).astype(np.uint8)

    return 1 - binarized_image

##Orienter image

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pytesseract

# Fonction pour détecter et corriger l'orientation de l'image en utilisant la détection de lignes de Hough
def correct_image_orientation_using_lines(image, angle_threshold=1.0):
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un flou pour réduire le bruit
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Détecter les bords avec Canny
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

    # Appliquer la transformation de Hough pour détecter les lignes
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # Si aucune ligne n'est détectée, retourner l'image sans modification
    if lines is None:
        print("Aucune ligne détectée.")
        return image, 0

    # Calculer l'angle moyen des lignes détectées
    angles = []
    for line in lines:
        rho, theta = line[0]
        angle = np.degrees(theta) - 90  # L'angle est mesuré à partir de l'axe horizontal
        angles.append(angle)

    # Calculer l'angle moyen
    mean_angle = np.mean(angles)
    print(f"Angle moyen détecté : {mean_angle:.2f}°")

    # Si l'angle est supérieur au seuil, on considère qu'il faut corriger l'orientation
    if abs(mean_angle) > angle_threshold:
        print(f"L'image est inclinée de {mean_angle} degrés, la rotation est nécessaire.")
        rows, cols = image.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), mean_angle, 1)
        image = cv2.warpAffine(image, M, (cols, rows))
    else:
        print(f"L'image est correctement orientée (angle {mean_angle:.2f}°).")

    return image, mean_angle

def correct_image_orientation_with_tesseract(image):
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = sauvola_binarization(gray, window_size=35, k=0.08, R=128)



    # Utiliser pytesseract pour obtenir l'orientation de l'image
    osd = pytesseract.image_to_osd(gray)

    # Extraire l'angle de rotation de l'OSD (Orientation and Script Detection)
    rotation_angle = int(osd.split('\n')[0].split(':')[1].strip())
    print(f"Angle de rotation détecté par Tesseract : {rotation_angle}°")

    # Appliquer la rotation si nécessaire
    if rotation_angle != 0:
        rows, cols = image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), rotation_angle, 1)
        image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

    return image, rotation_angle

# Exemple d'utilisation
# import os

# # Supposons que vous avez déjà chargé l'image comme suit :
# image_path = os.path.join(extracted_etiquette_folder, image_files[2])
# image = cv2.imread(image_path)
# # image = sauvola_binarization(image, window_size=35, k=0.08, R=128)


# # Exemple d'utilisation
# import os

# # Supposons que vous avez déjà chargé l'image comme suit :
# image_path = os.path.join(extracted_etiquette_folder, image_files[0])
# image = cv2.imread(image_path)

# # Effectuer les rotations de 45° cumulées à chaque itération
# for i in range(4):  # On fait pivoter l'image de 45° à chaque itération, pour un total de 360°
#     rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)  # Rotation de 90° chaque fois
#     image = rotated_image  # Met à jour l'image pour la prochaine itération

#     corrected_image, angle = correct_image_orientation_with_tesseract(rotated_image)  # Seuil de 0.2° pour éviter des corrections inutiles

#     print(f"Rotation {90*(i+1)}° - Angle moyen calculé : {angle:.2f}°")

#     # Afficher l'image avant et après correction
#     display_before_after(rotated_image, corrected_image, f"Avant correction {90*(i+1)}°", f"Après correction {90*(i+1)}°")


# cv2.destroyAllWindows()


##Blur

In [ ]:
def apply_blur(image, blur_kernel=(5, 5)):
    # Appliquer un flou gaussien léger
    blurred_image = cv2.GaussianBlur(image, blur_kernel, 0)

    return blurred_image

##Scale

In [ ]:
def scale_image_to_size(image, target_width=600, target_height=300):
    resized_image = cv2.resize(image, (target_width, target_height))
    return resized_image

## Remove snow

In [ ]:
import numpy as np
import cv2
from skimage.morphology import remove_small_objects
import matplotlib.pyplot as plt

def remove_snow(binary_image, min_size=500):
    """
    Supprimer le bruit de neige dans une image binaire en supprimant les petits objets.

    Parameters:
    - image : ndarray
        L'image binaire (0 et 1).
    - min_size : int
        La taille minimale d'un objet à conserver (en nombre de pixels).

    Returns:
    - image : ndarray
        L'image avec le bruit de neige supprimé.
    """

    binary_image = binary_image.astype(bool)
    # Supprimer les petits objets
    cleaned_image = remove_small_objects(binary_image, min_size=min_size)

    # Convertir l'image traitée de retour à l'échelle
    return (cleaned_image).astype(np.uint8)




## Fill holes

In [31]:
import numpy as np
from scipy import ndimage

import numpy as np
from scipy import ndimage

def fill_holes(image, max_hole_size=None, structure=None, origin=0):
    """
    Remplir les trous dans une image binaire, avec une limite sur la taille des trous à remplir.

    Parameters:
    - image : array_like
        L'image binaire avec des objets et des trous à remplir.
    - max_hole_size : int, optionnel
        La taille maximale des trous à remplir (en pixels). Si None, tous les trous seront remplis.
    - structure : array_like, optionnel
        L'élément structurant utilisé dans la dilatation. Si None, un élément structurant par défaut est utilisé.
    - origin : int, optionnel
        Position de l'élément structurant. Par défaut, `origin=0`.

    Returns:
    - ndarray
        L'image avec les trous remplis.
    """
    # Assurez-vous que l'image d'entrée est un tableau numpy binaire
    image = np.asarray(image)

    # Vérifier si l'image est binaire, sinon la convertir
    if image.max() > 1:
        _, image = cv2.threshold(image, 127, 1, cv2.THRESH_BINARY)

    # Convertir l'image en type booléen (True pour l'objet, False pour le fond)
    image = image.astype(bool)

    # Identifier les trous (zones de fond connectées entourées par des objets)
    inverted_image = ~image
    labeled_holes, num_features = ndimage.label(inverted_image, structure=structure)

    # Calculer la taille de chaque trou
    hole_sizes = np.bincount(labeled_holes.ravel())

    # Créer une image pour les trous qui respectent la contrainte de taille
    allowed_holes = np.zeros_like(labeled_holes, dtype=bool)

    for i in range(1, len(hole_sizes)):  # Ignore le fond (label 0)
        if max_hole_size is None or hole_sizes[i] <= max_hole_size:
            allowed_holes[labeled_holes == i] = True

    # Remplir uniquement les trous valides
    filled_image = image | allowed_holes

    # Convertir l'image booléenne en image binaire (0 ou 1)
    return filled_image.astype(np.uint8)


#Pre traitement

In [34]:
def process_image_1(image):
    return image

# Function to apply all transformations to a given image
def process_image_2(image):

    # 1. Appliquer process_1 sur l'image
    image = process_image_1(image)

    # 2. Orienter l'image (corriger la rotation)
    # oriented_image = correct_image_orientation_using_lines(image)

    # 3. Appliquer le deskewing
    deskewed_image = deskew_image(image)

    # 4. Appliquer le flou léger
    blurred_image = apply_blur(deskewed_image)

    # 5. Redimensionner l'image
    resized_image = scale_image_to_size(blurred_image)

    return resized_image

def process_image_3(image):

    # deskewed_image = deskew_image(image)

    resized_image = scale_image_to_size(image)

    sauvola_image = sauvola_binarization(resized_image, window_size=35, k=0.08, R=128)

    return sauvola_image

def process_image_4(image):
    # 1. Appliquer process_3 sur l'image
    image = process_image_3(image)

    removed_snow = remove_snow(image,min_size=5)

    filled = fill_holes(removed_snow,500)


    return filled


In [35]:
input_dir = '/content/drive/MyDrive/Project/Deep Learning/etiquettes_pre_traitees/unprocessed_etiquettes'
output_dir_2 = '/content/drive/MyDrive/Project/Deep Learning/etiquettes_pre_traitees/etiquettes_process_2'
output_dir_3 = '/content/drive/MyDrive/Project/Deep Learning/etiquettes_pre_traitees/etiquettes_process_3'
output_dir_4 = '/content/drive/MyDrive/Project/Deep Learning/etiquettes_pre_traitees/etiquettes_process_4'

def save_image(image, directory, filename):
    path = os.path.join(directory, filename)
    cv2.imwrite(path, image)

for directory in [output_dir_2, output_dir_3, output_dir_4]:
    if not os.path.exists(directory):
        os.makedirs(directory)

image_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)
    processed_2 = process_image_2(image)
    save_image(processed_2, output_dir_2, image_file)
    processed_3 = process_image_3(image)
    save_image((1-processed_3)*255, output_dir_3, image_file)
    processed_4 = process_image_4(image)
    save_image((1-processed_4)*255, output_dir_4, image_file)

print("Done")

Done
